In [19]:
import random
import matplotlib.pyplot as plt 
from scipy.spatial import Delaunay
import numpy as np
from csp import *
import time


class ColorMap:
    def __init__(self, N):
        self.N = N
        self.points = np.array([[random.uniform(0,1),random.uniform(0,1)] for _ in range(self.N)])
        self.tri = Delaunay(self.points)

    def obtain_neighbours(self, pindex):
        neighbors = list()
        for simplex in self.tri.simplices:
            if pindex in simplex:
                neighbors.extend([simplex[i] for i in range(len(simplex)) if simplex[i] != pindex])

        return list(set(neighbors))

    def create_csp_neighbour_dictionary(self):
        D = {}
        for i in range(self.N):
            D[i] = self.obtain_neighbours(i)
        return D
    
    def display_points(self):
        plt.triplot(self.points[:,0], self.points[:,1], self.tri.simplices)
        plt.plot(self.points[:,0], self.points[:,1], 'o')
        plt.show()

print("First we show how slow min conflicts is. ")
print("We only run 10 experiments with 5 'countries': ")
t0 = time.time()
for i in range(10):
    color_map = ColorMap(5)
    DD = color_map.create_csp_neighbour_dictionary()
    mccsp = MapColoringCSP(list('RGB'), DD)
    solution = min_conflicts(mccsp)
t1 = time.time()

print("min_conflicts", t1 - t0)

First we show how slow min conflicts is. 
We only run 10 experiments with 5 'countries': 
min_conflicts 61.7578341960907


Sometimes, when we run the above code, the problem generater picks a nice problem and min_conflicts completes very quickly. This is seldom the case. Typically, we get the above situation: 60 seconds!

We get an enermous speed-up by using backtracking search...

In [20]:
t0 = time.time()
for i in range(30):
    color_map = ColorMap(15)
    DD = color_map.create_csp_neighbour_dictionary()
    mccsp = MapColoringCSP(list('RGB'), DD)
    solution = backtracking_search(mccsp)
t1 = time.time()

print("backtracking_search", t1 - t0)

t0 = time.time()
for i in range(30):
    color_map = ColorMap(15)
    DD = color_map.create_csp_neighbour_dictionary()
    mccsp = MapColoringCSP(list('RGB'), DD)
    solution = backtracking_search(mccsp, order_domain_values=lcv, select_unassigned_variable=mrv, inference=forward_checking)
t1 = time.time()

print("backtracking_search with forward_checking", t1 - t0)


t0 = time.time()
for i in range(30):
    color_map = ColorMap(15)
    DD = color_map.create_csp_neighbour_dictionary()
    mccsp = MapColoringCSP(list('RGB'), DD)
    solution = backtracking_search(mccsp, order_domain_values=lcv, select_unassigned_variable=mrv, inference=mac)
t1 = time.time()

print("backtracking_search with mac", t1 - t0)

backtracking_search 2.0141031742095947
backtracking_search with forward_checking 0.4799063205718994
backtracking_search with mac 0.7199668884277344
